In [1]:
import sqlite3
import plyvel
from pathlib import Path
import re
import json
from functions.runHealth import runHealth
# from functions.dynamic_analysis import Dynamic
import requests

In [2]:
# path : Path = Path('/home/ndanner_plp/OpenWPM/crawl-data/datadir8000')
path : Path = Path('/mnt/crawls/datadir-0-10')
con : sqlite3.Connection = sqlite3.connect( str(path.joinpath("crawl-data.sqlite")) )
db = plyvel.DB( str(path.joinpath("leveldb")) )
%load_ext sql
%sql sqlite:///{path}/crawl-data.sqlite

'Connected: @/mnt/crawls/datadir-0-10/crawl-data.sqlite'

In [3]:
n, f = runHealth(con)
print(f"total visits: {n}, failed/incomplete visits: {f}. Success percentage: {round(100* (1 - f/n)) }%")

total visits: 10, failed/incomplete visits: 3. Success percentage: 70%


In [4]:
%%sql
SELECT incomplete_visits.visit_id, site_visits.site_url
FROM incomplete_visits
INNER JOIN site_visits 
ON incomplete_visits.visit_id = site_visits.visit_id

 * sqlite:////mnt/crawls/datadir-0-10/crawl-data.sqlite
Done.


visit_id,site_url
4006986725082397,http://www.a-msedge.net
5338836672544214,http://www.akamaiedge.net


In [5]:
%%sql
SELECT symbol, count(DISTINCT visit_id) AS n_sites, count(*) as n
FROM javascript
GROUP BY symbol
ORDER BY symbol

 * sqlite:////mnt/crawls/datadir-0-10/crawl-data.sqlite
Done.


symbol,n_sites,n
CanvasRenderingContext2D.arc,1,1
CanvasRenderingContext2D.beginPath,1,2
CanvasRenderingContext2D.fill,1,1
CanvasRenderingContext2D.fillStyle,1,1
CanvasRenderingContext2D.lineTo,1,2
CanvasRenderingContext2D.lineWidth,1,1
CanvasRenderingContext2D.moveTo,1,2
CanvasRenderingContext2D.stroke,1,2
CanvasRenderingContext2D.strokeStyle,1,1
HTMLCanvasElement.getContext,4,15


In [30]:
%%sql
SELECT DISTINCT symbol
FROM javascript
WHERE symbol NOT LIKE "window.navigator.mimeTypes%" AND symbol NOT LIKE "window.navigator.plugins%"

 * sqlite:////mnt/crawls/datadir-0-10/crawl-data.sqlite
   sqlite:////mnt/crawls/datadir-0-100/crawl-data.sqlite
Done.


symbol


In [7]:
%%sql
SELECT COUNT(*)
FROM javascript
LIMIT 100

 * sqlite:////mnt/crawls/datadir-0-10/crawl-data.sqlite
Done.


COUNT(*)
828


In [ ]:
# %load_ext sql
# symbols = {}
# for path in Path("/home/ndanner_plp/OpenWPM/crawl-data").iterdir():
#     if path.is_dir() and re.match("^datadir" ,path.name):
#         %sql sqlite:///{path}/crawl-data.sqlite
#         result = %sql SELECT DISTINCT(symbol) FROM javascript
#         if result:
#             for symbol in result.DataFrame()['symbol']:
#                 if symbol in symbols:
#                     symbols[symbol].append(path.name)
#                 else:
#                     symbols[symbol] = [path.name]
# print( json.dumps( symbols, indent= 4 ) )

OfflineAudioContext

In [ ]:
%%sql
SELECT *
FROM javascript
LIMIT 10

In [ ]:
%%sql
WITH a AS (
SELECT visit_id,url, COUNT(*) as n
FROM http_responses
WHERE content_hash <> ""
GROUP BY visit_id,url
ORDER BY n DESC
)
SELECT a.visit_id, site_visits.site_url, a.url, a.n
FROM a
INNER JOIN site_visits
ON a.visit_id = site_visits.visit_id
LIMIT 20


In [ ]:
%%sql
WITH a AS (
SELECT visit_id,url, COUNT( DISTINCT content_hash ) as unique_content_hashs
FROM http_responses
WHERE content_hash <> ""
GROUP BY visit_id,url
ORDER BY unique_content_hashs DESC
)
SELECT a.visit_id, site_visits.site_url, a.url, a.unique_content_hashs
FROM a
INNER JOIN site_visits
ON a.visit_id = site_visits.visit_id
LIMIT 10


In [ ]:
%%sql
SELECT content_hash
FROM http_responses
WHERE visit_id = "5244274392029786" AND url = "https://biptolyla.com/auW.ZNyzQu2r9FkyZnT/9U6gbE2C5mlFS/WvQT9RNPDlE/4OMrjrk/0HN/Cc0N0/MLT/g/yNOkTTQK1ZJPnEB-1KcV2-hiaRbO2/5/lGS/WeQs9iNIDeEZ4rMtjTkS0/NVCI0R0UMWT/gwy/OlTQQ/1R"
GROUP BY content_hash

In [ ]:
import difflib
from pprint import pprint


x = difflib.Differ()
str1 = str( db.get(b"338359dbbf86b03f293159f6673b33f7f87a0ab0a6930df27bfc69582b824748"), encoding="utf-8")
str2 = str( db.get(b"339f38f18297db8afa9cdd0f873676e520ab736af47643d682ac158900f78913"), encoding="utf-8")
cmp = x.compare( str1.splitlines(keepends=True), str2.splitlines(keepends=True)  )
for line in filter(lambda s : s[0] == "+" or s[0] == "-", cmp ):
    print(line,end = "\n\n\n\n")


In [ ]:
%%sql
SELECT *
FROM site_visits
WHERE visit_id = "7805288562003358" OR visit_id = "6723308048261410"

In [ ]:
r = requests.get("http://localhost:8000", params={})

In [ ]:
r.text